In [1]:
# IMPORTS
############################################################################################################################################
import pandas as pd
import numpy as np
from pandas import json_normalize
import requests
import json
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib #FOR HOLDOUT

In [2]:
# API URL Base Checker
url = 'https://api.sportmonks.com/v3/football/fixtures/16924773?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic'

headers = {
    'Authorization': 'Bearer YOUR_API_KEY',
    'Content-Type': 'application/json'
}

# Send a GET request to the API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON
    data = response.json()
    
    # Print the filtered DataFrame
    print(data)
else:
    print(f"Error: {response.status_code}, Response: {response.text}")



{'data': {'id': 16924773, 'sport_id': 1, 'league_id': 8, 'season_id': 17420, 'stage_id': 77448322, 'group_id': None, 'aggregate_id': None, 'round_id': 201980, 'state_id': 5, 'venue_id': 230, 'name': 'Liverpool vs Burnley', 'starting_at': '2021-01-21 20:00:00', 'result_info': 'Burnley won after full-time.', 'leg': '1/1', 'details': None, 'length': 90, 'placeholder': False, 'has_odds': True, 'has_premium_odds': False, 'starting_at_timestamp': 1611259200}, 'subscription': [{'meta': {'trial_ends_at': None, 'ends_at': '2025-07-03 19:28:14', 'current_timestamp': 1750979094}, 'plans': [{'plan': 'Charles Camarena Custom Plan 2', 'sport': 'Football', 'category': 'Custom'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 3359, 'remaining': 2998, 'requested_entity': 'Fixture'}, 'timezone': 'UTC'}


In [3]:
# GET SEASONS IN ORDER BY DATE
############################################################################################################################################

season_ids = [
    2, 3, 7, 9, 10, 11, 12, 13, 6397, 
    12962, 16036, 17420, 18378, 19734, 21646 
    # ,23614 # 2024 season
    # ,24255, 24256, 24261, 24262, 24263 #These Correspond to very early seasons
]
dates_df = []
# Loop over each season ID and construct the URL
for season_id in season_ids:
    url = f'https://api.sportmonks.com/v3/football/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic'
    headers = {
        'Authorization': 'Bearer YOUR_API_KEY',
        'Content-Type': 'application/json'
    }
    
    # Send a GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful (status code 200 means success)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        
        # Pretty-print the JSON data
        pretty_data = json.dumps(data, indent=4)
    else:
        print(f"Error: {response.status_code}")
    json_data = json.loads(pretty_data)
    # Use json_normalize to extract id and name from nested structure
    df = json_normalize(json_data['data'], sep='_')
    
    # Selecting only the required fields
    df_filt = df[['id', 'starting_at']]
    df_filt.columns = ['season', 'starting_at']
    dates_df.append(df_filt)
date_df = pd.concat(dates_df, ignore_index=True)
date_df = date_df.sort_values(by = 'starting_at', ignore_index=True)
date_df['index_column'] = date_df.index
print(date_df)


        
    

    season starting_at  index_column
0       11  2009-08-15             0
1        2  2010-08-14             1
2        9  2011-08-13             2
3        7  2012-08-18             3
4        3  2013-08-17             4
5       12  2014-08-16             5
6       10  2015-08-08             6
7       13  2016-08-13             7
8     6397  2017-08-11             8
9    12962  2018-08-10             9
10   16036  2019-08-09            10
11   17420  2020-09-12            11
12   18378  2021-08-13            12
13   19734  2022-08-05            13
14   21646  2023-08-11            14


In [4]:
# GET FIXTURE DATA
# FINAL DF INCLUDES TEAM STATS, OPPONENT STATS, OTHER FEATURES
############################################################################################################################################
combined_df = pd.read_csv('combined_df.csv') #Just Headers for our df we are gonna build
full_final = []


# Loop over each season ID and construct the URL
count = 0
for season_id in season_ids:
    count += 1
    # url = f'https://api.sportmonks.com/v3/football/teams/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic'
    url = f'https://api.sportmonks.com/v3/football/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic'
    headers = {
        'Authorization': 'Bearer YOUR_API_KEY',
        'Content-Type': 'application/json'
    }
    
    # Send a GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful (status code 200 means success)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        
        # Pretty-print the JSON data
        pretty_data = json.dumps(data, indent=4)
    else:
        print(f"Error: {response.status_code}")

    
    json_data = json.loads(pretty_data)
    # Use json_normalize to extract id and name from nested structure
    df = json_normalize(json_data['data'], sep='_')
    
    # Selecting only the required fields
    df_filtered = df[['id', 'name']]
    
    # Rename columns for clarity
    df_filtered.columns = ['id', 'name']

    
    # Define the API endpoint and any parameters or headers if needed
    url = f'https://api.sportmonks.com/v3/football/rounds/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic'
    
    headers = {
        'Authorization': 'Bearer YOUR_API_KEY',
        'Content-Type': 'application/json'
    }
    
    # Send a GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful (status code 200 means success)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        
        # Pretty-print the JSON data
        pretty_data = json.dumps(data, indent=4)
        # print(pretty_data)
    else:
        print(f"Error: {response.status_code}")
    
    json_data = json.loads(pretty_data)
    # Use json_normalize to extract id and name from nested structure
    df = json_normalize(json_data['data'], sep='_')
    # Selecting only the required fields
    df_filtered = df[['id', 'starting_at', 'ending_at', 'season_id']]
    
    # Rename columns for clarity
    df_filtered.columns = ['round_id', 'start', 'end', 'season']
    df_round = df_filtered.copy()
    df_gws = df_filtered.sort_values(by=['start'])
    df_gws.rename(columns={'start': 'start_of_gw'}, inplace=True)

    df_gws.reset_index(drop=True, inplace=True)
    df_gws.index = df_gws.index + 1  # Add 1 to each index
    
    # Reset the index and keep the current index (1-38) as a new column named 'gwnum'
    df_gws = df_gws.reset_index()
    
    # Rename the index column to 'gwnum'
    df_gws.rename(columns={'index': 'gwnum'}, inplace=True)
    
    # Define the API endpoint and any parameters or headers if needed
    url = f'https://api.sportmonks.com/v3/football/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic&include=fixtures.participants'
    
    headers = {
        'Authorization': 'Bearer YOUR_API_KEY',
        'Content-Type': 'application/json'
    }
    
    # Send a GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful (status code 200 means success)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        if count == 1:
            pretty_data = json.dumps(data, indent=4)
            # print(pretty_data)
        # Assuming df_round is your DataFrame with a column named 'round_id'
        round_ids = df_round['round_id'].tolist()  # Convert the 'round_id' column to a list
        
        # Get the fixtures from the data
        fixtures = data['data']['fixtures']
        
        # Filter fixtures by checking if the 'round_id' is in the round_ids list
        filtered_fixtures = [fixture for fixture in fixtures if fixture['round_id'] in round_ids]



        # Now, flatten the participants from the filtered fixtures
        df = json_normalize(
            filtered_fixtures,
            record_path='participants',  # Only pull data from participants
            meta=['id', 'round_id', 'name', 'starting_at'],  # Include 'name' and 'starting_at'
            record_prefix='participant_'
        )

        
        # # Now, flatten the participants from the filtered fixtures
        # df = json_normalize(
        #     filtered_fixtures, 
        #     record_path='participants',  # Only pull data from participants
        #     meta=['id', 'round_id'],
        #     record_prefix='participant_'
        # )
        
        # Select relevant columns
        df = df[['id', 'round_id', 'name', 'starting_at', 'participant_id', 'participant_meta.location', 'participant_meta.winner']]
        # Rename columns for clarity
        df.rename(columns={
            'starting_at': 'start',
            'participant_id': 'team_id',
            'participant_meta.location': 'home/away',
            'participant_meta.winner': 'winner',
        }, inplace=True)
        
        # Group by fixture to determine ties (sum of winners per fixture)
        df_grouped = df.groupby('id')['winner'].sum().reset_index()
        
        # Calculate result points
        df['points'] = df['winner'].apply(lambda x: 3 if x else 0)
        df.loc[df['id'].isin(df_grouped[df_grouped['winner'] == 0]['id']), 'points'] = 1  # For ties
        df_result = df[['id', 'round_id', 'name', 'start', 'team_id', 'points', 'home/away']]

    else:
        print(f"Error: {response.status_code}")
    
    df_result.sort_values(by='round_id')
    ############################################################################################################################################

    # print(df_gws.head())
    full_df = pd.merge(df_gws, df_result, on='round_id')
    # print(full_df.columns)
    ############################################################################################################################################

    full_df = full_df[['season', 'id', 'gwnum', 'name', 'start', 'team_id', 'points', 'home/away']]
    
    # Create a new column 'opponent' by grouping by 'id' and assigning the opponent team_id
    full_df['opponent'] = full_df.groupby('id')['team_id'].transform(lambda x: x.iloc[::-1].values)
    full_df['opponent_pts'] = full_df.groupby('id')['points'].transform(lambda x: x.iloc[::-1].values)
    
    home_df = full_df.copy()

    # Define a function to calculate the cumulative sum of scores for both the team and the opponent
    def cumulative_sum_team_and_opponent(row, df):
        # Calculate cumulative sum of points for the team
        team_points_sum = df[(df['team_id'] == row['team_id']) & (df['gwnum'] <= row['gwnum'])]['points'].sum()
        
        # Calculate cumulative sum of points for the opponent
        opponent_points_sum = df[(df['team_id'] == row['opponent']) & (df['gwnum'] <= row['gwnum'])]['points'].sum()
        
        return team_points_sum, opponent_points_sum
    
    # Apply the function and create two new columns for team's and opponent's cumulative points
    home_df[['sum_smaller_gw', 'opponent_sum']] = home_df.apply(
        lambda row: pd.Series(cumulative_sum_team_and_opponent(row, home_df)), axis=1
    )

    
    index_df = home_df.copy()
    index_df = index_df.reset_index(drop=True)
    index_df.index.name = 'order' 
    # team_33_df.sort_values(by=['gwnum', 'id'])
    index_df =  index_df.sort_values(by=['team_id','gwnum'])
    index_df['form'] = index_df.groupby('team_id')['sum_smaller_gw'].transform(lambda x: x - x.shift(5))
    # Step 1: Calculate the form for each team as we did before
    # Create a temporary DataFrame with form calculation for each team
    team_form_df = index_df[['team_id', 'gwnum', 'sum_smaller_gw']].copy()
    team_form_df['team_form'] = team_form_df.groupby('team_id')['sum_smaller_gw'].transform(lambda x: x - x.shift(5))
    
    # Step 2: Rename columns in the temporary DataFrame to indicate it's the opponent's form
    team_form_df = team_form_df.rename(columns={'team_id': 'opponent', 'team_form': 'opponent_form', 'gwnum': 'gwnum_opponent'})
    
    # Step 3: Merge the opponent's form back into the main DataFrame based on opponent ID and game week
    index_df = index_df.merge(team_form_df[['opponent', 'gwnum_opponent', 'opponent_form']],
                              left_on=['opponent', 'gwnum'],
                              right_on=['opponent', 'gwnum_opponent'],
                              how='left')
    
    # Step 4: Drop the auxiliary column 'gwnum_opponent' as it's no longer needed
    index_df = index_df.drop(columns=['gwnum_opponent'])
    
    # Display the resulting DataFrame with the new 'opponent_form' column
    # print(index_df.head(10))

    url = f'https://api.sportmonks.com/v3/football/seasons/{season_id}?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic&include=fixtures.scores'

    headers = {
        'Authorization': 'Bearer YOUR_API_KEY',
        'Content-Type': 'application/json'
    }
    
    # Send a GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful (status code 200 means success)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        # Pretty-print the JSON data
        pretty_data = json.dumps(data, indent=4)
    # Extract the fixtures
    fixtures = data["data"]["fixtures"]
    
    # Create a list to store the data for the DataFrame
    current_scores_data = []
    
    # Iterate over each fixture to get the "CURRENT" scores
    for fixture in fixtures:
        for score in fixture["scores"]:
            if score["description"] == "CURRENT":
                current_scores_data.append({
                    "fixture_id": fixture["id"],
                    "team": score["score"]["participant"],
                    "goals": score["score"]["goals"]
                })
    
    # Convert to a DataFrame
    df_current_scores = pd.DataFrame(current_scores_data)
    
    # Pivot the data to have home and away goals in separate columns
    df_pivoted = df_current_scores.pivot(index='fixture_id', columns='team', values='goals').reset_index()
    
    # Rename columns to make them more descriptive
    df_pivoted.columns = ['fixture_id', 'away_goals', 'home_goals']
    
    # Display the DataFrame
    # print(df_pivoted)
    merged = index_df.merge(df_pivoted, left_on = 'id', right_on = 'fixture_id', how='inner', suffixes=('',''))




    final_df = merged.copy()
    # print(final_df)
    full_final.append(final_df)
    
    full_final_df = pd.concat(full_final, ignore_index=True)
    if season_id == 21646:
        last_season_list = merged['id'].unique().tolist()

# Create a list of unique values from the "id" column
fixture_list = full_final_df['id'].unique().tolist()

# Print the list (optional)
# print(fixture_list)
print(full_final_df.head())

    




   season    id  gwnum                                  name  \
0       2   465      1        Aston Villa vs West Ham United   
1       2  5504      2   West Ham United vs Bolton Wanderers   
2       2  5452      3  Manchester United vs West Ham United   
3       2  5313      4            West Ham United vs Chelsea   
4       2  5102      5         Stoke City vs West Ham United   

                 start  team_id  points home/away  opponent  opponent_pts  \
0  2010-08-14 14:00:00        1       0      away        15             3   
1  2010-08-21 14:00:00        1       0      home        16             3   
2  2010-08-28 16:30:00        1       0      away        14             3   
3  2010-09-11 14:00:00        1       0      home        18             3   
4  2010-09-18 11:45:00        1       1      away        26             1   

   sum_smaller_gw  opponent_sum  form  opponent_form  fixture_id  away_goals  \
0               0             3   NaN            NaN         465        

In [5]:
#RANDOM DATE FILTER TO AVOID NA/S
# Convert 'start' to datetime if it isn't already
full_final_df['start'] = pd.to_datetime(full_final_df['start'])

# Filter for fixtures after a specific date
filtered_df = full_final_df[(full_final_df['start'] > '2017-08-01')]

# Get unique fixture IDs
fix_list = filtered_df['fixture_id'].unique().tolist()

# # Optional: print number and first few
# print(f"Total fixtures after 2015-08-01: {len(fix_list)}")
# filtered_df = full_final_df[~full_final_df['season'].isin([19734, 21646])]

In [ ]:
# # Bookmakers to include
# bookmaker_ids = {1, 2, 3}

# # Hold results
# all_data = []
# no_data_fixtures = []

# # Loop over each fixture
# for i, fixture_id in enumerate(fix_list, start=1):
    
#     if i in {1, 10, 100, 400, 1000, 2000}:
#         print(f"\n📍Checkpoint — Processing fixture #{i} (fixture_id: {fixture_id})")

#     url = f'https://api.sportmonks.com/v3/football/odds/pre-match/fixtures/{fixture_id}/markets/1?api_token=GSoPMqoZ5e1lPovCnMOtOGlGmsREDwUP17vg54phbLQEX4OGNGDT7IDmF1ic&select=bookmaker_id,probability,label,fixture_id'

#     response = requests.get(url)

#     if response.status_code == 200:
#         data = response.json()

#         if 'data' in data and data['data']:
#             entries = data['data']
#             df = pd.DataFrame(entries)

#             # Filter only bookmaker 1, 2, 3
#             df = df[df['bookmaker_id'].isin(bookmaker_ids)]

#             if df.empty:
#                 print(f"❌ No data for fixture {fixture_id} from bookmakers 1,2,3")
#                 no_data_fixtures.append(fixture_id)
#                 continue

#             # Map labels
#             label_map = {'1': 'win_p', 'X': 'draw_p', '2': 'loss_p'}
#             df['label'] = df['label'].map(label_map)

#             # Drop any unrecognized labels
#             df = df[df['label'].notna()]

#             # Convert percentages if needed
#             df['probability'] = df['probability'].replace('%', '', regex=True).astype(float)

#             # Pivot: rows = bookmaker_id, columns = label
#             pivot_df = df.pivot_table(index='bookmaker_id', columns='label', values='probability', aggfunc='first')

#             # Average across available bookmakers
#             averaged = pivot_df.mean(skipna=True)

#             # If all values are NaN, skip fixture
#             if averaged.isna().all():
#                 print(f"⚠️ All 3 bookmakers missing data for fixture {fixture_id}")
#                 no_data_fixtures.append(fixture_id)
#                 continue

#             # Add to main list
#             row = {
#                 'fixture_id': fixture_id,
#                 'win_p': averaged.get('win_p', None),
#                 'draw_p': averaged.get('draw_p', None),
#                 'loss_p': averaged.get('loss_p', None),
#             }
#             all_data.append(row)

#         else:
#             print(f"❌ No 'data' in API response for fixture {fixture_id}")
#             no_data_fixtures.append(fixture_id)

#     else:
#         print(f"❌ Failed to fetch fixture {fixture_id} — status {response.status_code}")
#         no_data_fixtures.append(fixture_id)

# # Final DataFrame
# result_df = pd.DataFrame(all_data)

# # Show result
# print("\n🎯 Final Result DataFrame:")
# print(result_df)

# # Show skipped fixtures
# print(f"\n🛑 Fixtures with no data: {len(no_data_fixtures)}")
# print(no_data_fixtures)


In [7]:
odds_df = pd.read_csv('od.csv')
odds_df = odds_df.rename(columns={
    'fixture_id': 'id',
    'win_p': 'home_win',
    'draw_p': 'draw',
    'loss_p': 'away_win'
})
odds_df['home_win'] = odds_df['home_win']/ 100
odds_df['draw'] = odds_df['draw']/ 100
odds_df['away_win'] = odds_df['away_win']/ 100

# df_merged = full_final_df[['name', 'id']].merge(odds_df
# print(full_final_df.head())
# print(odds_df.head())

In [8]:
# HOME STRENGTH FEATURE PREVIOUS SEASONS
############################################################################################################################################
merged_full_df = full_final_df.merge(date_df, on='season', how='inner', suffixes=('', ''))

# Define a function to calculate home and away points for a team in the previous season
def calculate_home_strength(team_id, previous_season_df):
    # Filter for games where the team played at home and calculate the total points
    home_points = previous_season_df[(previous_season_df['team_id'] == team_id) & (previous_season_df['home/away'] == 'home')]['points'].sum()
    
    # Filter for games where the team played away and calculate the total points
    away_points = previous_season_df[(previous_season_df['team_id'] == team_id) & (previous_season_df['home/away'] == 'away')]['points'].sum()
    
    # Calculate the custom feature score as the difference
    return home_points - away_points

# Initialize a column for the home strength score
merged_full_df['home_strength_score'] = None

# Loop through each unique season (index_column)
for index_value in merged_full_df['index_column'].unique():
    # Check if there is a previous season
    if index_value == 0:
        # For the first season, set home_strength_score to 0
        merged_full_df.loc[merged_full_df['index_column'] == index_value, 'home_strength_score'] = 0
        continue
    
    # Define the current and previous season data based on index_column
    current_season_df = merged_full_df[merged_full_df['index_column'] == index_value]
    previous_season_df = merged_full_df[merged_full_df['index_column'] == index_value - 1]
    
    # Calculate the home strength score for each team in the previous season
    team_home_strength = {}
    for team_id in previous_season_df['team_id'].unique():
        team_home_strength[team_id] = calculate_home_strength(team_id, previous_season_df)
    
    # Assign the home strength score to each row of the current season for the corresponding team
    def assign_home_strength(row):
        return team_home_strength.get(row['team_id'], 0)  # Default to 0 if no previous season data
    
    merged_full_df.loc[merged_full_df['index_column'] == index_value, 'home_strength_score'] = current_season_df.apply(assign_home_strength, axis=1)

# Print the updated DataFrame with home_strength_score



In [9]:
# HOME STRENGTH FEATURE CURRENT SEASON
############################################################################################################################################
# Define a function to calculate cumulative home strength up to the current game week within the same season
def calculate_cumulative_home_strength(row, df):
    # Filter for games up to the current game week within the same season for the given team
    team_id = row['team_id']
    season = row['season']
    gw = row['gwnum']
    
    # Get all games up to the current game week within the same season for the team
    past_games = df[(df['season'] == season) & (df['team_id'] == team_id) & (df['gwnum'] < gw)]
    
    # Calculate cumulative home and away points
    home_games = past_games[past_games['home/away'] == 'home']
    away_games = past_games[past_games['home/away'] == 'away']
    
    # Calculate the total points for home and away games
    home_points = home_games['points'].sum()
    away_points = away_games['points'].sum()
    
    # Calculate the average points per game for home and away games
    home_avg = home_points / len(home_games) if len(home_games) > 0 else 0
    away_avg = away_points / len(away_games) if len(away_games) > 0 else 0
    
    # Calculate the custom feature score as the difference
    return home_avg - away_avg

# Initialize a new column for the cumulative home strength score within the current season
merged_full_df['home_strength_score2'] = merged_full_df.apply(calculate_cumulative_home_strength, axis=1, df=merged_full_df)

# Print the updated DataFrame with both home_strength_score and home_strength_score2
# print(merged_full_df.head(10))


In [10]:
# GD FEATURE
############################################################################################################################################
merged_full_df['goals_for'] = 0
merged_full_df['goals_against'] = 0

# Define a function to calculate cumulative goals for and goals against
def calculate_goals(row, df):
    team_id = row['team_id']
    season = row['season']
    gwnum = row['gwnum']
    home_or_away = row['home/away']
    
    # Filter for matches within the same season and before the current game week for the team
    past_matches = df[(df['season'] == season) & 
                      (df['gwnum'] < gwnum) & 
                      (df['team_id'] == team_id)]

    # Calculate goals_for
    goals_for = past_matches.apply(
        lambda x: x['home_goals'] if x['home/away'] == 'home' else x['away_goals'], axis=1
    ).sum()
    
    # Calculate goals_against
    goals_against = past_matches.apply(
        lambda x: x['away_goals'] if x['home/away'] == 'home' else x['home_goals'], axis=1
    ).sum()
    
    return pd.Series([goals_for, goals_against])

# Apply the function to each row
merged_full_df[['goals_for', 'goals_against']] = merged_full_df.apply(calculate_goals, axis=1, df=merged_full_df)
merged_full_df['gd'] = merged_full_df['goals_for'] - merged_full_df['goals_against']


In [12]:
# GW 15 - 38 FILTER
############################################################################################################################################
# print(full_final_df.head(10))
full_final_df = merged_full_df

# Initialize an empty list to collect DataFrames for each gameweek
combined_dfs = []

# Get the maximum gameweek number
max_gw = full_final_df['gwnum'].max()

############################################################################################################################################
for season_id in season_ids:
    for n in range(15, max_gw + 1):
        # Get data for the current and previous gameweeks
        current_season_df = full_final_df[full_final_df['season'] == season_id].copy()

        gw_n_df = current_season_df[current_season_df['gwnum'] == n].copy()
        gw_n_minus1_df = current_season_df[current_season_df['gwnum'] == n - 1].copy()

        # Ensure gw_n_minus1_df has unique team_id rows
        gw_n_minus1_team = gw_n_minus1_df[['team_id', 'sum_smaller_gw', 'form']].drop_duplicates(subset=['team_id'])
        gw_n_minus1_team = gw_n_minus1_team.rename(columns={
            'sum_smaller_gw': 'sum_smaller_gw_team_prev_gw',
            'form': 'form_team_prev_gw'
        })
        
        # Merge team data from the previous gameweek
        gw_n = gw_n_df.merge(gw_n_minus1_team, on='team_id', how='left')

        # Ensure unique opponent entries in previous gameweek
        gw_n_minus1_opp = gw_n_minus1_df[['team_id', 'sum_smaller_gw']].drop_duplicates(subset=['team_id']).rename(
            columns={'team_id': 'opponent', 'sum_smaller_gw': 'sum_smaller_gw_opponent_prev_gw'}
        )
        
        # Merge opponent data from the previous gameweek
        gw_n = gw_n.merge(gw_n_minus1_opp, left_on='opponent', right_on='opponent', how='left')

        # Append processed data
        combined_dfs.append(gw_n)

# Concatenate all DataFrames
combined_df = pd.concat(combined_dfs, ignore_index=True)
# Select rows where any column is NaN
nan_entries = combined_df[combined_df.isna().any(axis=1)]

# Filter rows with any NaN entries and get unique 'season' values
nan_seasons = combined_df[combined_df.isna().any(axis=1)]['season'].unique()

# Print the unique seasons with NaN entries
print("Unique seasons with NaN entries:", nan_seasons)

# print(nan_entries)

# Remove rows with any NaN values
# print(combined_df)
combined_df = combined_df.dropna()

Unique seasons with NaN entries: []


In [14]:
combined_df.to_csv('allfeat.csv')
dfcurr = pd.read_csv('allfeat.csv')


In [17]:
####PLAY AROUND###### (Looking at both home and away fixture)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Step 1: Prepare the data (already partially done)
df = combined_df.copy()

# Convert 'points' into three classes: 2 = win (3 points), 1 = draw (1 point), 0 = loss (0 points)
df['outcome'] = df['points']  # Use 'points' directly (3 -> win, 1 -> draw, 0 -> loss)
df['outcome'] = df['points'].map({3: 2, 1: 1, 0: 0})  # Map 3 to 2, 1 to 1, and 0 to 0


# Encode 'home/away' as a binary variable
label_encoder = LabelEncoder()
df['home/away'] = label_encoder.fit_transform(df['home/away'])
# df = df[df['home/away'] == 1]  # Only keep rows for home team

# Add home-specific features
df['home_strength_feature'] = df.apply(lambda row: row['home_strength_score'] if row['home/away'] == 1 else 0, axis=1)
df['home_strength_feature2'] = df.apply(lambda row: row['home_strength_score2'] if row['home/away'] == 1 else 0, axis=1)




# Step 2: Split into training and holdout sets
# Exclude specified seasons from training
# exclude_seasons = [18378, 21646, 19734]  # List of seasons to exclude
exclude_seasons = [21646]  # List of seasons to exclude
df = df[~df['season'].isin(exclude_seasons)].copy()


# season_of_interest = 17420  # Future season for testing
season_of_interest = 19734  # Future season for testing



holdout_df = df[df['season'] == season_of_interest]  # Test data
train_df = df[df['season'] != season_of_interest]  # Training data

# Step 3: Define features and target for model training
# features = ['gd', 'form_team_prev_gw', 'sum_smaller_gw_team_prev_gw', 'sum_smaller_gw_opponent_prev_gw', 'home/away', 'home_strength_feature2', 'home_strength_feature', 'opponent_form']
features = ['gd', 'form_team_prev_gw', 'sum_smaller_gw_team_prev_gw', 'sum_smaller_gw_opponent_prev_gw', 'home/away', 'home_strength_feature2', 'home_strength_feature']
X_train = train_df[features]
y_train = train_df['outcome']

# Step 4: Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 5: Train the multinomial logistic regression model
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model = LogisticRegression(solver='lbfgs', max_iter=200)

model.fit(X_train_scaled, y_train)



# View feature weights (coefficients) for each class (Loss, Draw, Win)
import pandas as pd

feature_weights_df = pd.DataFrame(
    model.coef_,
    columns=features,
    index=['Loss', 'Draw', 'Win']
)

print("Feature weights for each outcome class:")
print(feature_weights_df.T)  # Transpose for readability





# Step 6: Predict probabilities on the holdout set
X_holdout = holdout_df[features]
y_holdout = holdout_df['outcome']
X_holdout_scaled = scaler.transform(X_holdout)

holdout_df = holdout_df.copy()  # Ensure it's a full copy before editing

holdout_df.loc[:, 'Model_Prob_Loss'] = y_holdout_pred_proba[:, 0]
holdout_df.loc[:, 'Model_Prob_Draw'] = y_holdout_pred_proba[:, 1]
holdout_df.loc[:, 'Model_Prob_Win'] = y_holdout_pred_proba[:, 2]



# Step 7: Merge with betting site odds (from df_pivot)


holdout_df = holdout_df.merge(odds_df, on='id', how='left')


# Fix relative difference calculations based on team location
holdout_df['Rel_Diff_Win'] = holdout_df.apply(
    lambda row: row['Model_Prob_Win'] - (row['home_win'] if row['home/away'] == 1 else row['away_win']),
    axis=1
)

holdout_df['Rel_Diff_Loss'] = holdout_df.apply(
    lambda row: row['Model_Prob_Loss'] - (row['away_win'] if row['home/away'] == 1 else row['home_win']),
    axis=1
)

holdout_df['Rel_Diff_Draw'] = holdout_df['Model_Prob_Draw'] - holdout_df['draw']


def get_best_bet(row, base_bet=100, threshold=0.05):
    rel_diffs = {
        'Win': row['Rel_Diff_Win'],
        'Loss': row['Rel_Diff_Loss'],
        'Draw': row['Rel_Diff_Draw']
    }
    best_outcome = max(rel_diffs, key=rel_diffs.get)
    relative_diff = rel_diffs[best_outcome]
    bet_amount = base_bet if relative_diff > threshold else 0
    return best_outcome, bet_amount



# Apply betting logic to each fixture
bets = holdout_df.apply(get_best_bet, axis=1)
holdout_df['Bet_Placed'] = bets.apply(lambda x: x[0])
holdout_df['Bet_Amount'] = bets.apply(lambda x: x[1])

# Step 10: Calculate returns for each fixture
holdout_df['Return'] = 0
holdout_df['Profit/Loss'] = 0  # Track profit or loss for each game
# print(holdout_df.head())
count = 0
holdout_df['Return'] = 0.0
holdout_df['Profit/Loss'] = 0.0

for index, row in holdout_df.iterrows():
    if row['Bet_Placed'] == 'Win':
        odds = 1 / row['home_win']
    elif row['Bet_Placed'] == 'Loss':
        odds = 1 / row['away_win']
    elif row['Bet_Placed'] == 'Draw':
        odds = 1 / row['draw']
    else:
        odds = 0  # No bet placed
        count += 1

    # Check if the bet matches the actual outcome
    actual_outcome = {2: 'Win', 1: 'Draw', 0: 'Loss'}[row['outcome']]
    is_successful_bet = row['Bet_Placed'] == actual_outcome


    # Calculate return and profit/loss
    holdout_df.loc[index, 'Return'] = row['Bet_Amount'] * odds if is_successful_bet else 0
    holdout_df.loc[index, 'Profit/Loss'] = holdout_df.loc[index, 'Return'] - row['Bet_Amount']

# Step 11: Calculate total profit/loss from all bets
total_profit = holdout_df['Profit/Loss'].sum()
# print(f"Total profit/loss from betting: ${total_profit:.2f}")

# Step 12: Save detailed betting results to a CSV file
holdout_df.to_csv("betting_results2.csv", index=False)
# print("Betting results saved to 'betting_results2.csv'")
condensed_checker_df=holdout_df[['season', 'id', 'gwnum', 'name', 'start', 'outcome', 'Model_Prob_Loss', 'Model_Prob_Draw', 'Model_Prob_Win', 'away_win', 'draw', 'home_win', 'Bet_Placed', 'Return', 'Profit/Loss']]
condensed_checker_df = condensed_checker_df.sort_values(by = ['id', 'start'])
condensed_checker_df.to_csv("play_checker.csv", index=False)
# print(condensed_checker_df['profit'].sum())

Feature weights for each outcome class:
                                     Loss      Draw       Win
gd                              -0.064697 -0.047178  0.111874
form_team_prev_gw                0.016569  0.007358 -0.023927
sum_smaller_gw_team_prev_gw     -0.367376 -0.002731  0.370107
sum_smaller_gw_opponent_prev_gw  0.435980 -0.050217 -0.385763
home/away                       -0.217272 -0.024400  0.241672
home_strength_feature2          -0.035483  0.020248  0.015235
home_strength_feature           -0.031662  0.026923  0.004738


In [18]:
# BETTING SIMULATOR CORRECT(Only looking at home team)



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Step 1: Prepare the data (already partially done)
df = combined_df.copy()

# Convert 'points' into three classes: 2 = win (3 points), 1 = draw (1 point), 0 = loss (0 points)
df['outcome'] = df['points']  # Use 'points' directly (3 -> win, 1 -> draw, 0 -> loss)
df['outcome'] = df['points'].map({3: 2, 1: 1, 0: 0})  # Map 3 to 2, 1 to 1, and 0 to 0


# Encode 'home/away' as a binary variable
label_encoder = LabelEncoder()
df['home/away'] = label_encoder.fit_transform(df['home/away'])
df = df[df['home/away'] == 1]  # Only keep rows for home team

# Add home-specific features
df['home_strength_feature'] = df.apply(lambda row: row['home_strength_score'] if row['home/away'] == 1 else 0, axis=1)
df['home_strength_feature2'] = df.apply(lambda row: row['home_strength_score2'] if row['home/away'] == 1 else 0, axis=1)



# 11   17420  2020-09-12            11
# 12   18378  2021-08-13            12
# 13   19734  2022-08-05            13
# 14   21646



# Step 2: Split into training and holdout sets
# Exclude specified seasons from training
# exclude_seasons = [18378, 21646, 19734]  # List of seasons to exclude
exclude_seasons = [21646]  # List of seasons to exclude
df = df[~df['season'].isin(exclude_seasons)].copy()




# exclude_seasons = [18378]  # List of seasons to exclude
# df = df[~df['season'].isin(exclude_seasons)].copy()
# season_of_interest = 17420  # Future season for testing
season_of_interest = 19734  # Future season for testing


# exclude_seasons = [18378, 17420]  # List of seasons to exclude
# df = df[~df['season'].isin(exclude_seasons)].copy()
# season_of_interest = 21646  # Future season for testing
# season_of_interest = 18378  # Future season for testing




holdout_df = df[df['season'] == season_of_interest]  # Test data
train_df = df[df['season'] != season_of_interest]  # Training data

# Step 3: Define features and target for model training
# features = ['gd', 'form_team_prev_gw', 'sum_smaller_gw_team_prev_gw', 'sum_smaller_gw_opponent_prev_gw', 'home/away', 'home_strength_feature2', 'home_strength_feature', 'opponent_form']
features = ['gd', 'form_team_prev_gw', 'sum_smaller_gw_team_prev_gw', 'sum_smaller_gw_opponent_prev_gw', 'home/away', 'home_strength_feature2', 'home_strength_feature']
X_train = train_df[features]
y_train = train_df['outcome']

# Step 4: Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 5: Train the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X_train_scaled, y_train)



# View feature weights (coefficients) for each class (Loss, Draw, Win)
import pandas as pd

feature_weights_df = pd.DataFrame(
    model.coef_,
    columns=features,
    index=['Loss', 'Draw', 'Win']
)

print("Feature weights for each outcome class:")
print(feature_weights_df.T)  # Transpose for readability





# Step 6: Predict probabilities on the holdout set
X_holdout = holdout_df[features]
y_holdout = holdout_df['outcome']
X_holdout_scaled = scaler.transform(X_holdout)

# Predict probabilities for win, draw, and loss
y_holdout_pred_proba = model.predict_proba(X_holdout_scaled)
holdout_df['Model_Prob_Loss'] = y_holdout_pred_proba[:, 0]
holdout_df['Model_Prob_Draw'] = y_holdout_pred_proba[:, 1]
holdout_df['Model_Prob_Win'] = y_holdout_pred_proba[:, 2]


# Step 7: Merge with betting site odds (from df_pivot)


holdout_df = holdout_df.merge(odds_df, on='id', how='left')


# Step 8: Calculate relative differences
holdout_df['Rel_Diff_Home'] = holdout_df['Model_Prob_Win'] - holdout_df['home_win']
holdout_df['Rel_Diff_Away'] = holdout_df['Model_Prob_Loss'] - holdout_df['away_win']
holdout_df['Rel_Diff_Draw'] = holdout_df['Model_Prob_Draw'] - holdout_df['draw']

def get_best_bet(row, base_bet=100, threshold=0.15):
    # Calculate relative differences
    rel_diffs = {
        'Win': row['Rel_Diff_Home'],
        'Loss': row['Rel_Diff_Away'],
        'Draw': row['Rel_Diff_Draw']
    }
    # Get the best outcome and relative difference
    best_outcome = max(rel_diffs, key=rel_diffs.get)
    relative_diff = rel_diffs[best_outcome]
    
    # Place a bet only if relative difference exceeds the threshold
    bet_amount = base_bet if relative_diff > threshold else 0
    
    return best_outcome, bet_amount


# Apply betting logic to each fixture
bets = holdout_df.apply(get_best_bet, axis=1)
holdout_df['Bet_Placed'] = bets.apply(lambda x: x[0])
holdout_df['Bet_Amount'] = bets.apply(lambda x: x[1])

# Step 10: Calculate returns for each fixture
holdout_df['Return'] = 0
holdout_df['Profit/Loss'] = 0  # Track profit or loss for each game
# print(holdout_df.head())
count = 0
for index, row in holdout_df.iterrows():
    if row['Bet_Placed'] == 'Win':
        odds = 1 / row['home_win']
    elif row['Bet_Placed'] == 'Loss':
        odds = 1 / row['away_win']
    elif row['Bet_Placed'] == 'Draw':
        odds = 1 / row['draw']
    else:
        odds = 0  # No bet placed
        count += 1

    # Check if the bet matches the actual outcome
    actual_outcome = {2: 'Win', 1: 'Draw', 0: 'Loss'}[row['outcome']]
    is_successful_bet = row['Bet_Placed'] == actual_outcome


    # Calculate return and profit/loss
    holdout_df.loc[index, 'Return'] = row['Bet_Amount'] * odds if is_successful_bet else 0
    holdout_df.loc[index, 'Profit/Loss'] = holdout_df.loc[index, 'Return'] - row['Bet_Amount']

# Step 11: Calculate total profit/loss from all bets
total_profit = holdout_df['Profit/Loss'].sum()
print(f"Total profit/loss from betting: ${total_profit:.2f}")

# Step 12: Save detailed betting results to a CSV file
holdout_df.to_csv("betting_results2.csv", index=False)
# print("Betting results saved to 'betting_results2.csv'")
condensed_checker_df=holdout_df[['season', 'id', 'gwnum', 'name', 'start', 'outcome', 'Model_Prob_Loss', 'Model_Prob_Draw', 'Model_Prob_Win', 'away_win', 'draw', 'home_win', 'Bet_Placed', 'Return', 'Profit/Loss']]
condensed_checker_df = condensed_checker_df.sort_values(by = ['id', 'start'])
condensed_checker_df.to_csv("play_checker.csv", index=False)
# print(condensed_checker_df['profit'].sum())

Feature weights for each outcome class:
                                     Loss      Draw       Win
gd                              -0.090980 -0.063224  0.154204
form_team_prev_gw                0.051787 -0.006268 -0.045518
sum_smaller_gw_team_prev_gw     -0.396058  0.016431  0.379628
sum_smaller_gw_opponent_prev_gw  0.406427 -0.049427 -0.356999
home/away                        0.000000  0.000000  0.000000
home_strength_feature2          -0.039051  0.024046  0.015005
home_strength_feature           -0.034802  0.033978  0.000824
Total profit/loss from betting: $349.76


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/var/folders/7r/gd_3jt3n5lqcwpyx538q_jjw0000gn/T/ipykernel_48389/2488344692.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdout_df['Model_Prob_Loss'] = y_holdout_pred_proba[:, 0]
/var/folders/7r/gd_3jt3n5lqcwpyx538q_jjw0000gn/T/ipykernel_48389/2488344692.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c